In [1]:
#先获取收藏里面所有图片的推荐，然后去重，爬取所有推荐的收藏夹

In [155]:
import requests
import re
import json
import time
import threading
from urllib.parse import quote,unquote
import os
#==========获取收藏夹所有图片==========
#style0 返回图片id style1 返回图片数据
def get_photo_list(url,style):
    #==========获取数据==========
    def get_data(url):
        headers ={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36"
            }
        s = requests.Session()
        picture_list = []
        r_json = s.get(url,headers=headers)
        r_json = r_json.text
        false = False
        true = True
        r_json = r_json.replace('\n', '')
        r_json = eval(r_json)
        
        data = r_json["data"]
        
        return data
    #==========获取total==========
    def get_total(api_url):
        data = get_data(api_url)
        total = data["total"]
        return total
    #==========处理数据，返回列表==========
    def processing_data(data):
        total = data["total"]
        object_list = data["object_list"]
        temp_list = []
        photo_list = []
        for object_child in object_list:
            photo_id = object_child["id"]
            photo_name = object_child["msg"]
            photo_url = object_child["photo"]["path"]
            
            photo_url = photo_url.replace(".jpg",".thumb.1000_0.jpg")
            photo_url = photo_url.replace(".jpeg",".thumb.1000_0.jpeg")
            photo_url = photo_url.replace(".png",".thumb.1000_0.png")
            photo_url = photo_url.replace(".gif",".thumb.1000_0.gif")
            photo_url = photo_url.replace("gif_jpeg","gif") #修正一个奇怪的数据
            
            favorite_count = object_child["favorite_count"]
            photo_list.append([photo_id,photo_name,photo_url,favorite_count])
            temp_list.append(photo_id)
        return [temp_list,photo_list]
    def get_photo_id_list(url):
        photo_list = []
        id  = re.findall(r"id=(\d+)",url)[0]
        api_url = "https://www.duitang.com/napi/blog/list/by_album/?album_id={0}&limit=100&include_fields=top_comments%2Cis_root%2Csource_link%2Cbuyable%2Croot_id%2Cstatus%2Clike_count%2Clike_id%2Csender%2Creply_count&start={1}".format(id,0)
        total = get_total(api_url)
        for start in range(0,total,100):
            api_url = "https://www.duitang.com/napi/blog/list/by_album/?album_id={0}&limit=100&include_fields=top_comments%2Cis_root%2Csource_link%2Cbuyable%2Croot_id%2Cstatus%2Clike_count%2Clike_id%2Csender%2Creply_count&start={1}".format(id,start)
            data = get_data(api_url)
            photo_list.extend(processing_data(data)[0])
        return photo_list
    def get_photo_infos(url):
        photo_infos = []
        id  = re.findall(r"id=(\d+)",url)[0]
        api_url = "https://www.duitang.com/napi/blog/list/by_album/?album_id={0}&limit=100&include_fields=top_comments%2Cis_root%2Csource_link%2Cbuyable%2Croot_id%2Cstatus%2Clike_count%2Clike_id%2Csender%2Creply_count&start={1}".format(id,0)
        total = get_total(api_url)
        for start in range(0,total,100):
            api_url = "https://www.duitang.com/napi/blog/list/by_album/?album_id={0}&limit=100&include_fields=top_comments%2Cis_root%2Csource_link%2Cbuyable%2Croot_id%2Cstatus%2Clike_count%2Clike_id%2Csender%2Creply_count&start={1}".format(id,start)
            data = get_data(api_url)
            photo_infos.extend(processing_data(data)[1])
        return photo_infos
    if style == 0 :
        return get_photo_id_list(url)
    elif style == 1 :
        
        return get_photo_infos(url)
#==========获取图片页面内的专辑==========
def get_albums(id):
    url = "https://www.duitang.com/napi/blog/detail/?blog_id={}&include_fields=tags%2Crelated_albums%2Crelated_albums.covers%2Croot_album%2Cshare_links_2%2Cextra_links%2Cicon_description%2Croot_id".format(id)
    headers ={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36"
                }
    s = requests.Session()
    r_json = unquote(s.get(url,headers=headers).text)
    false = False
    true = True
    r_json = r_json.replace('\n', '')
    r_json = eval(r_json)
    data = r_json["data"]
    related_albums = data["related_albums"]
    album_list = []
    for album in related_albums:
        id = album["id"]
        name = album["name"]
        count = album["count"]
        favorite_count = album["favorite_count"]
        album_list.append({id:[name,count,favorite_count,favorite_count/count]})
    return album_list
#==========列表去重==========
def deduplication(all_album_list):
    new_list = []
    for album in all_album_list:
        if album not in new_list:
            new_list.append(album)
    return new_list
#==========获得所有图集id==========
#相册图片数量小于num,相册分数大于score 得分=收藏数/图片数 
#推荐次数越多该相册内容与本相册相关性越高
def get_ids(num,score,lower_recommended_times): 
    from collections import Counter
    ids = []
    #==========输入id返回相册名字==========
    def get_item(id):
        for i in all_album:
            if id == list(i.keys())[0]:
                return i
    
    
    #==========筛选收藏数与图片数==========
    for i in all_album:
        for key,v in i.items():
            favorite_count = v[2]
            picture_num = v[1]
            score_ = v[3]
            if picture_num < num and score_> score:
                ids.append(key)
    ids = Counter(ids)
    k_y_ids = []
    new_ids = []
    #==========筛选推荐次数大于lower_recommended_times==========
    for k,y in ids.items():
        if y >= lower_recommended_times:
            
            k_y_ids.append({y:get_item(k)})
#             print(k,y)
#             print(get_item(k))
#             print("https://www.duitang.com/album/?id=%s"%k)
    def temp_def(arr):
        recommended_times = list(arr.keys())[0]
        return recommended_times
    k_y_ids.sort(key = temp_def,reverse=True) #根据推荐次数排序
    for i in k_y_ids:
        recommended_times = list(i.keys())[0]
        id = list(i.values())[0]
        id = list(id.keys())[0]
        new_ids.append(id)
        print(i)
        print("https://www.duitang.com/album/?id=%s"%id)
    return new_ids
#==========开始下载图片==========
#返回失败列表
#lower_favorite_count
def download_all_pictures(path,all_pictures,lower_favorite_count):
    
    def get_exist_list(path):
        exist_list = os.listdir(path)
        exist_id = []
        for name in exist_list:
            id = re.findall(r".*_(\d+)\.",name)[0]
            exist_id.append(id)
        return set(exist_id)
    if not os.path.exists(path):
        os.makedirs(path)
    exist_list = get_exist_list(path)
    failure = []
    temp = 0
    print(exist_list)
    for i in all_pictures:
        try:
            picture_url = i[2]
            favorite_count = i[3]
            id = i[0]
            print(id)
            if (str(id) in exist_list):
                print("已存在，跳过"+str(i))
                temp += 1
                continue
            elif (favorite_count < lower_favorite_count):
                temp += 1
                continue
            s = requests.Session()
            code = s.get(picture_url)
            suffix = re.findall(r'.*\.(.*)',picture_url)[0]
            name = path + str(favorite_count)+"_"+str(id)+"."+suffix
            open(name, 'wb').write(code.content) # 将内容写入图片
            temp += 1
            percent=temp/len(all_pictures)*100
            print("\r{:.0f}%".format(percent),end = " ")
            print(str(i))
        except:
            print("下载失败:"+str(i))
            
            failure.append(i)
            time.sleep(1)

gif


In [156]:
#==========获得所有图片里面所有的推荐==========
input_url = "https://www.duitang.com/album/?id=101105862"
photo_id_list = get_photo_list(input_url,0)
all_album = []
i = 0
for id in photo_id_list:
    try:
        i += 1
        percent=i/len(photo_id_list)*100
        print("\r{:.0f}%".format(percent),end = "")

        all_album.extend(get_albums(id))
    except:
        print("失败:"+str(id))

#==========整理，按id排序==========
def return_id(arr):
    id = list(arr.keys())[0]
    return id
all_album.sort(key =return_id,reverse=True)

with open("all_album.txt","w",encoding='utf-8') as f:
    f.write(str(all_album))
    


100%

In [160]:
#==========获取所有图集id==========
#相册图片数量小于num,相册分数大于score 得分=收藏数/图片数 
#推荐次数越多该相册内容与本相册相关性越高
ids = get_ids(1000,0.1,3)

{12: {101242780: ['默认专辑', 59, 10, 0.1694915254237288]}}
https://www.duitang.com/album/?id=101242780
{8: {101707430: ['猫，表情', 542, 65, 0.11992619926199262]}}
https://www.duitang.com/album/?id=101707430
{8: {101105862: ['hhh', 142, 116, 0.8169014084507042]}}
https://www.duitang.com/album/?id=101105862
{6: {102155003: ['表情包', 113, 36, 0.3185840707964602]}}
https://www.duitang.com/album/?id=102155003
{6: {99431338: ['嗯', 591, 280, 0.47377326565143824]}}
https://www.duitang.com/album/?id=99431338
{6: {90349731: ['表情包', 341, 48, 0.14076246334310852]}}
https://www.duitang.com/album/?id=90349731
{4: {82496891: ['萝莉控的表情包', 911, 1401, 1.5378704720087815]}}
https://www.duitang.com/album/?id=82496891
{3: {96803864: ['嘿嘿', 17, 6, 0.35294117647058826]}}
https://www.duitang.com/album/?id=96803864
{3: {95104287: ['该沙雕时就沙雕丫', 229, 83, 0.3624454148471616]}}
https://www.duitang.com/album/?id=95104287


In [20]:
#==========移除id==========
remove_list = [94705997,96915673,100386184,4302573,76482796,94579360,71219707]
for i in remove_list:
    try:
        ids.remove(i)
    except:
        pass

In [162]:
#==========获取列表里所有图片==========
all_pictures = []
i = 0
for id in ids:
    atlas_url = "https://www.duitang.com/album/?id=%s"%id
    temp_list = get_photo_list(atlas_url,style = 1)
    all_pictures.extend(temp_list)
    print("图集大小:"+str(len(all_pictures)),end = "")
#     print(all_pictures)
    all_pictures=deduplication(all_pictures)
    i += 1
    percent=i/len(ids)*100
    print(" {:.0f}%".format(percent))
def temp_def(all_pictures):
    return all_pictures[3]
all_pictures.sort(key = temp_def,reverse=True)

图集大小:46 11%
图集大小:555 22%
图集大小:643 33%
图集大小:746 44%
图集大小:1245 56%
图集大小:1559 67%
图集大小:2328 78%
图集大小:2341 89%
图集大小:2541 100%


In [163]:
#==========开始下载==========
failure = download_all_pictures("./堆糖图片/h/",all_pictures,lower_favorite_count=3)
print(failure)

{'1226147968', '1194423205', '1259017216', '1259576609', '1194422971', '1259019889', '1161920772', '1259017070', '1259017258', '1259019906', '1259017012', '1287279555', '1233857144'}
1215581162
0% [1215581162, '表情包', 'https://c-ssl.duitang.com/uploads/item/201608/02/20160802222411_BfKuL.thumb.1000_0.jpeg', 1200]
707025694
0% [707025694, '表情包', 'https://c-ssl.duitang.com/uploads/item/201611/06/20161106090534_fNcQP.thumb.1000_0.jpeg', 1175]
1262534406
0% [1262534406, '表情包', 'https://c-ssl.duitang.com/uploads/item/201801/21/20180121093341_VATEL.thumb.1000_0.jpeg', 956]
1281508641
0% [1281508641, '表情包', 'https://c-ssl.duitang.com/uploads/item/201803/17/20180317140643_udvmy.thumb.1000_0.jpg', 938]
1133322427
0% [1133322427, '动漫 头像 女头', 'https://c-ssl.duitang.com/uploads/item/201604/20/20160420000932_vP2VE.thumb.1000_0.jpeg', 888]
1279344210
0% [1279344210, '表情包', 'https://c-ssl.duitang.com/uploads/item/201611/29/20161129122354_rNcZy.thumb.1000_0.jpeg', 811]
1133322165
0% [1133322165, '表情包',

2% [1278325709, '小可爱表情包', 'https://c-ssl.duitang.com/uploads/item/201808/11/20180811163055_muwld.thumb.1000_0.jpeg', 260]
723591615
2% [723591615, '表情包 基', 'https://c-ssl.duitang.com/uploads/item/201703/04/20170304115859_zCh4n.thumb.1000_0.jpeg', 259]
1133324875
2% [1133324875, '表情包 基', 'https://c-ssl.duitang.com/uploads/item/201703/04/20170304115859_zCh4n.thumb.1000_0.jpeg', 259]
747723275
2% [747723275, '表情包 蔡文姬 王者荣耀', 'https://c-ssl.duitang.com/uploads/item/201610/24/20161024204141_VFmJy.thumb.1000_0.jpeg', 254]
1277589732
3% [1277589732, '。。。', 'https://c-ssl.duitang.com/uploads/item/201705/12/20170512073448_fYFGJ.thumb.1000_0.jpeg', 251]
713208838
3% [713208838, '头像（__葡萄丶收集', 'https://c-ssl.duitang.com/uploads/item/201603/12/20160312171024_vjaWR.thumb.1000_0.png', 251]
753752743
3% [753752743, '小污婆的表情包─北方酱', 'https://c-ssl.duitang.com/uploads/item/201604/12/20160412223559_Qvsed.thumb.1000_0.jpeg', 246]
1233857144
已存在，跳过[1233857144, '#疯狂动物城#', 'https://c-ssl.duitang.com/uploads/ite

5% [1276644131, '小可爱表情包', 'https://c-ssl.duitang.com/uploads/item/201811/02/20181102123930_vneey.thumb.1000_0.jpeg', 172]
1279865368
5% [1279865368, '表情包', 'https://c-ssl.duitang.com/uploads/item/201908/19/20190819202945_xs5nZ.thumb.1000_0.jpeg', 168]
1286035667
5% [1286035667, '表情', 'https://c-ssl.duitang.com/uploads/item/202001/17/20200117133436_aceWG.thumb.1000_0.jpeg', 166]
1276658898
5% [1276658898, '表情包', 'https://c-ssl.duitang.com/uploads/item/201908/29/20190829115931_k3uVK.thumb.1000_0.jpeg', 166]
1223457399
5% [1223457399, '动漫表情包【文豪野犬】', 'https://c-ssl.duitang.com/uploads/blog/201611/20/20161120171242_VBy8L.thumb.1000_0.jpeg', 165]
1278554695
5% [1278554695, '[橘久收集] [动漫头像] [动态头像]', 'https://c-ssl.duitang.com/uploads/item/201410/08/20141008164416_VP3zk.thumb.1000_0.gif', 164]
1278382512
5% [1278382512, '群里存的', 'https://c-ssl.duitang.com/uploads/item/202001/29/20200129142206_cudof.thumb.1000_0.gif', 164]
1215374125
5% [1215374125, '表情包 整日沉迷小姐姐', 'https://c-ssl.duitang.com/upload

7% [1214926009, "小污婆的表情包o(〃'▽'〃)o", 'https://c-ssl.duitang.com/uploads/item/201611/14/20161114233534_kWfs2.thumb.1000_0.jpeg', 133]
1228937284
7% [1228937284, '＃滑稽＃动态＃表情包＃', 'https://c-ssl.duitang.com/uploads/item/201611/01/20161101180751_kSVYJ.thumb.1000_0.gif', 133]
1131299353
7% [1131299353, '动漫表情包', 'https://c-ssl.duitang.com/uploads/item/201809/22/20180922121708_zRNBX.thumb.1000_0.jpeg', 133]
1280398678
7% [1280398678, '奶饿', 'https://c-ssl.duitang.com/uploads/item/202001/21/20200121004840_rweye.thumb.1000_0.jpg', 131]
1277565746
7% [1277565746, '表情', 'https://c-ssl.duitang.com/uploads/item/201805/14/20180514101532_Yzn2W.thumb.1000_0.jpeg', 131]
713954060
7% [713954060, '小污婆的表情包', 'https://c-ssl.duitang.com/uploads/item/201604/11/20160411000543_23Fek.thumb.1000_0.jpeg', 131]
713915648
7% [713915648, '大海原', 'https://c-ssl.duitang.com/uploads/item/201508/01/20150801191003_zU5Wj.thumb.1000_0.jpeg', 131]
721062485
7% [721062485, '头像 头上长草的颜文字君', 'https://c-ssl.duitang.com/uploads/item/2

10% [1276288446, '这次是恋爱专场表情包', 'https://c-ssl.duitang.com/uploads/item/201910/19/20191019190236_ixgjt.thumb.1000_0.jpeg', 106]
747353139
10% [747353139, 'file:///C:/Users/lenovo/AppData/Roaming/Tencent/QQ/Temp/WI2PI78]A28_LUUW$42J[DE.jpg\n【胸口一滴奶茶】0℃兔子小麦草<0℃奶茶>  17:34:20', 'https://c-ssl.duitang.com/uploads/item/201704/30/20170430174437_zBSCt.thumb.1000_0.jpeg', 106]
1262526920
10% [1262526920, '群里存的', 'https://c-ssl.duitang.com/uploads/item/202001/07/20200107210445_jvxpx.thumb.1000_0.gif', 104]
1222310764
10% [1222310764, '北方栖姬表情 (´･ω･｀) 许某 收图 侵删 ( ´_ゝ｀) _(:3 」∠)_ @巧克力真姬', 'https://c-ssl.duitang.com/uploads/blog/201510/02/20151002080503_dzjtw.thumb.1000_0.jpeg', 104]
762460132
10% [762460132, '刀剑乱舞表情包 加州清光', 'https://c-ssl.duitang.com/uploads/item/201609/24/20160924211836_dYume.thumb.1000_0.jpeg', 104]
1279866256
下载失败:[1279866256, '表情包', 'https://c-ssl.duitang.com/uploads/item/201909/04/20190904183827_3Fc2c.thumb.1000_0.gif', 103]
1215265495
10% [1215265495, '表情包 至死方休 埃罗曼阿老师', 'https:/

...

76% [1279866108, '恋爱表情包', 'https://c-ssl.duitang.com/uploads/item/201804/16/20180416120004_uinxp.thumb.1000_0.jpeg', 5]
1278377359
76% [1278377359, '动漫表情包', 'https://c-ssl.duitang.com/uploads/item/202005/11/20200511162739_aqrbj.thumb.1000_0.png', 5]
1278374641
76% [1278374641, '动漫表情包 ', 'https://c-ssl.duitang.com/uploads/item/202006/27/20200627070627_ynkrk.thumb.1000_0.jpg', 5]
1278371593
76% [1278371593, '表情', 'https://c-ssl.duitang.com/uploads/item/202004/03/20200403071150_mnkby.thumb.1000_0.jpg', 5]
1278306332
76% [1278306332, '表情图', 'https://c-ssl.duitang.com/uploads/item/201806/28/20180628175405_HPzZC.thumb.1000_0.jpeg', 5]
1276684273
下载失败:[1276684273, '动漫表情包\n毁灭吧烦了自截\n', 'https://c-ssl.duitang.com/uploads/item/202007/18/20200718223925_Bf2ZV.thumb.1000_0.gif', 5]
1276650184
76% [1276650184, '表情包.表白', 'https://c-ssl.duitang.com/uploads/item/201908/12/20190812000905_qqsbu.thumb.1000_0.jpg', 5]
1276648282
76% [1276648282, '表情包', 'https://c-ssl.duitang.com/uploads/item/202003/12/20200

79% [1177527867, '。。。。', 'https://c-ssl.duitang.com/uploads/item/201903/20/20190320180739_tSRP4.thumb.1000_0.jpeg', 5]
1107407038
79% [1107407038, '未知出处', 'https://c-ssl.duitang.com/uploads/item/201907/24/20190724124830_mdrxg.thumb.1000_0.jpg', 5]
948941039
79% [948941039, '表情包', 'https://c-ssl.duitang.com/uploads/item/201804/27/20180427010746_zdwcy.thumb.1000_0.jpeg', 5]
787590258
79% [787590258, '简介真的好难写啊！这图是我从网上随便找的啊！', 'https://c-ssl.duitang.com/uploads/item/201707/18/20170718215908_uJwGy.thumb.1000_0.jpeg', 5]
787209156
79% [787209156, '简介不会填o(´^｀)o', 'https://c-ssl.duitang.com/uploads/item/201707/18/20170718123621_nyFBY.thumb.1000_0.jpeg', 5]
781144979
79% [781144979, '动漫+表情包 实用表情 小表情 萌萌哒 贱 蠢 暴走 (๑>\u0602<๑）[灵子]', 'https://c-ssl.duitang.com/uploads/item/201604/19/20160419221032_MzCKv.thumb.1000_0.jpeg', 5]
778390173
79% [778390173, '简介不会填o(´^｀)o', 'https://c-ssl.duitang.com/uploads/item/201707/04/20170704132415_ShAiu.thumb.1000_0.jpeg', 5]
768240933
79% [768240933, '北方栖姬', 'https

81% [1215375756, '援前三连', 'https://c-ssl.duitang.com/uploads/item/201808/06/20180806205048_cltob.thumb.1000_0.jpg', 4]
1215359150
81% [1215359150, '表情包~', 'https://c-ssl.duitang.com/uploads/item/201910/05/20191005200525_xT2nT.thumb.1000_0.jpeg', 4]
1215352164
81% [1215352164, '怎么了？', 'https://c-ssl.duitang.com/uploads/item/201706/15/20170615004904_LjQuZ.thumb.1000_0.jpeg', 4]
1215264035
81% [1215264035, '盗的', 'https://c-ssl.duitang.com/uploads/item/202002/18/20200218120150_pnjuw.thumb.1000_0.jpg', 4]
1215259018
81% [1215259018, '盗的', 'https://c-ssl.duitang.com/uploads/item/202003/24/20200324104520_kdhsg.thumb.1000_0.jpg', 4]
1215249019
81% [1215249019, 'get cure', 'https://c-ssl.duitang.com/uploads/item/201808/25/20180825180636_mqmqs.thumb.1000_0.jpg', 4]
1215245951
81% [1215245951, '表情包ಥ_ಥ', 'https://c-ssl.duitang.com/uploads/item/201902/28/20190228100626_Xyxvh.thumb.1000_0.jpeg', 4]
1214971669
81% [1214971669, '耶', 'https://c-ssl.duitang.com/uploads/item/201901/20/20190120233420_ebxuq

84% [1276647639, '❤️', 'https://c-ssl.duitang.com/uploads/item/201812/20/20181220224210_gpupc.thumb.1000_0.jpeg', 3]
1276646486
84% [1276646486, '1', 'https://c-ssl.duitang.com/uploads/item/202004/04/20200404021220_zfacf.thumb.1000_0.gif', 3]
1276644355
84% [1276644355, '百科全书', 'https://c-ssl.duitang.com/uploads/item/202003/31/20200331235209_qurvd.thumb.1000_0.jpg', 3]
1275865873
84% [1275865873, '眼睛小表情', 'https://c-ssl.duitang.com/uploads/item/201908/01/20190801004150_qecch.thumb.1000_0.jpg', 3]
1275865751
84% [1275865751, '眼睛小表情', 'https://c-ssl.duitang.com/uploads/item/201908/01/20190801004148_pylhi.thumb.1000_0.jpg', 3]
1275865689
84% [1275865689, '眼睛小表情', 'https://c-ssl.duitang.com/uploads/item/201908/01/20190801004216_zkyyg.thumb.1000_0.jpg', 3]
1275865635
84% [1275865635, '眼睛小表情', 'https://c-ssl.duitang.com/uploads/item/201908/01/20190801004213_zcblm.thumb.1000_0.jpg', 3]
1275865617
84% [1275865617, '眼睛小表情', 'https://c-ssl.duitang.com/uploads/item/201908/01/20190801004214_igrwl.

86% [1254730359, '动漫表情包', 'https://c-ssl.duitang.com/uploads/item/202006/16/20200616123044_jlssh.thumb.1000_0.jpg', 3]
1250716619
86% [1250716619, '盗的', 'https://c-ssl.duitang.com/uploads/item/202003/04/20200304072043_tpnjt.thumb.1000_0.jpg', 3]
1213263777
86% [1213263777, '是表情包鸭', 'https://c-ssl.duitang.com/uploads/item/201902/02/20190202074214_wtryf.thumb.1000_0.jpeg', 3]
1212994583
86% [1212994583, '表情包', 'https://c-ssl.duitang.com/uploads/item/201905/10/20190510192537_gqogs.thumb.1000_0.jpg', 3]
1181406639
86% [1181406639, '表情包', 'https://c-ssl.duitang.com/uploads/item/202001/26/20200126205558_emboi.thumb.1000_0.jpg', 3]
1171818378
86% [1171818378, 'dt小熊曲奇碎掉了', 'https://c-ssl.duitang.com/uploads/item/201906/20/20190620200137_pjuws.thumb.1000_0.jpg', 3]
1121587010
86% [1121587010, '明日方舟', 'https://c-ssl.duitang.com/uploads/item/201908/22/20190822092830_egoms.thumb.1000_0.jpg', 3]
1120961202
86% [1120961202, '诶。。？', 'https://c-ssl.duitang.com/uploads/item/201908/20/20190820224410_vts

In [5]:



with open ("all_album.txt","r") as f:
    all_album = f.read()

all_album = eval(all_album)
# print(all_album)

[{78524585: ['默认专辑', 2366, 3603, 1.5228233305156382]}, {102165001: ['See the star.', 150, 1, 0.006666666666666667]}, {74146092: ['图片寄存器', 13343, 576, 0.04316870269054935]}, {101449675: ['图集', 3739, 180, 0.04814121422840332]}, {85800547: ['杀死一只知更鸟.', 1450, 9901, 6.828275862068965]}, {58454335: ['头像', 477, 43, 0.09014675052410902]}, {85746498: ['头像。', 1511, 75, 0.04963600264725347]}, {87348867: ['少女味失足！', 1279, 230, 0.17982799061767005]}, {95485194: ['闺蜜', 6954, 1315, 0.18909979867702043]}, {83858562: ['默认专辑', 6174, 398, 0.0644638807904114]}, {96453377: ['两颗。闺蜜糖', 2901, 261, 0.08996897621509824]}, {94142937: ['哇！蜜蜜', 8367, 623, 0.07445918489303215]}, {95821852: ['闺蜜', 3502, 184, 0.052541404911479156]}, {100056104: ['默认专辑', 1, 0, 0.0]}, {83869405: ['          网女', 4472, 822, 0.1838103756708408]}, {94250020: ['［闺蜜］匣子里的秘密の三行情书', 2059, 1802, 0.875182127246236]}, {100502618: ['女生头像', 608, 38, 0.0625]}, {97269165: ['女生头像', 225, 53, 0.23555555555555555]}, {100306634: ['默认专辑', 1791, 180, 0.10050